In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import json
import re
import requests
import time

import swagger_client
from swagger_client.rest import ApiException

In [2]:
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += 'HIGH:!DH:!aNULL'
try:
    requests.packages.urllib3.contrib.pyopenssl.DEFAULT_SSL_CIPHER_LIST += 'HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

In [3]:
load_dotenv()
AEMET_KEY = os.getenv("KEY_AEMET")
configuration = swagger_client.Configuration()
configuration.api_key['api_key'] = AEMET_KEY
api_instance = swagger_client.ValoresClimatologicosApi(swagger_client.ApiClient(configuration))

In [4]:
df15 = pd.read_csv('data/Temp15-16_meteo.csv')
df16 = pd.read_csv('data/Temp16-17_meteo.csv')
df17 = pd.read_csv('data/Temp17-18_meteo.csv')

In [19]:
df15.head()

,Equipo Local,Goles,Ataque,Medio,Defensa,Goles_Acumulados,Equipo Visitante,Goles.1,Ataque.1,Medio.1,...,Resultado,cod_est,tmin,tmed,tmax,prec,presmin,presmax,velmedia,rachaviento
0,Málaga,0,75,76,75,0,Sevilla,0,79,80,...,1,6156X,"24,4","28,0","31,7","0,0","1010,4","1014,3","2,2","8,1"
1,Deportivo de La Coruña,0,79,74,74,0,Real Sociedad,0,77,77,...,1,1387,"16,2","19,8","23,3","5,8","1004,5","1010,4","3,3","13,1"
2,Espanyol,1,76,75,74,0,Getafe,0,73,75,...,2,0076,"23,5","26,2","28,8","0,05","1010,1","1018,0","2,2","8,3"
3,Atlético de Madrid,1,81,80,81,0,Las Palmas,0,73,71,...,2,3195,"17,4","24,6","31,7","0,0","934,1","942,7","2,5","11,7"
4,Rayo Vallecano,0,75,75,74,0,Valencia,0,79,80,...,1,3195,"17,4","24,6","31,7","0,0","934,1","942,7","2,5","11,7"


In [18]:
def change_prec(val):
    if val == 'Ip':
        return '0,05'
    elif val == '-':
        return '0,0'
    else:
        return val
    
df15.prec = df15.prec.apply(lambda x: change_prec(x))

In [8]:
est_meteo_mod15 = {
'Las Palmas': {'prov':'Las Palmas',
               'cod_est':'C659H'},
}

In [9]:
df15.loc[pd.isnull(df15.tmin), 'cod_est'] = df15[pd.isnull(df15.tmin)]['Equipo Local'].apply(lambda x: est_meteo_mod15[x]['cod_est'])

In [11]:
for index, row in df15[pd.isnull(df15.tmin)].iterrows():
    print(f'Obteniendo datos de la fila: {index}')
    fecha_ini = row.Día + 'T00:00:00UTC'
    fecha_fin = row.Día + 'T23:59:59UTC'
    estacion = row.cod_est
    
    try:
        respuesta_api = api_instance.climatologas_diarias_(fecha_ini, fecha_fin, estacion)
        url_respuesta = respuesta_api.datos
        response = requests.request("GET", url_respuesta)
        print(response)
        print('\n\n')
        meteo = json.loads(response.text)[0]
        
        try:
            few.loc[index, 'tmin'] = meteo['tmin']
        except:
            few.loc[index, 'tmin'] = '-'
        try:
            few.loc[index, 'tmed'] = meteo['tmed']
        except:
            few.loc[index, 'tmed'] = '-'
        try:
            few.loc[index, 'tmax'] = meteo['tmax']
        except:
            few.loc[index, 'tmax'] = '-'
        try:
            few.loc[index, 'prec'] = meteo['prec']
        except:
            few.loc[index, 'prec'] = '-'
        try:
            few.loc[index, 'presmin'] = meteo['presMin']
        except:
            few.loc[index, 'presmin'] = '-'
        try:
            few.loc[index, 'presmax'] = meteo['presMax']
        except:
            few.loc[index, 'presmax'] = '-'
        try:
            few.loc[index, 'velmedia'] = meteo['velmedia']
        except:
            few.loc[index, 'velmedia'] = '-'
        try:
            few.loc[index, 'rachaviento'] = meteo['racha']
        except:
            few.loc[index, 'rachaviento'] = '-'

        time.sleep(5)
    except ApiException as e:
        print ('Algo ha fallado')
    

Obteniendo datos de la fila: 261


ValueError: Invalid value for `datos`, must not be `None`

In [20]:
df15.to_csv('data/Temp15-16_meteo.csv', index=False)

In [ ]:
est_meteo = {
'Eibar': {'prov':'Gipuzkoa',
         'cod_est':'1050J'},
'Granada': {'prov':'Granada',
            'cod_est':'5530E'},
'Sporting de Gijón': {'prov':'Asturias',
                      'cod_est':'1207U'},
'Girona': {'prov':'Girona',
           'cod_est':'0367'},
'Espanyol': {'prov':'Barcelona',
             'cod_est':'0076'},
'Rayo Vallecano': {'prov':'Madrid',
                   'cod_est':'3195'},
'Athletic Club': {'prov':'Bizkaia',
                  'cod_est':'1082'},
'Alavés': {'prov':'Araba',
           'cod_est':'9091R'},
'Osasuna': {'prov':'Navarra',
            'cod_est':'9263D'},
'Deportivo de La Coruña': {'prov':'A Coruña',
                           'cod_est':'1387'},
'Sevilla': {'prov':'Sevilla',
            'cod_est':'5783'},
'Real Madrid': {'prov':'Madrid',
                'cod_est':'3195'},
'Leganés': {'prov':'Madrid',
            'cod_est':'3195'},
'Levante': {'prov':'Valencia',
            'cod_est':'8414A'},
'Barcelona': {'prov':'Barcelona',
              'cod_est':'0076'},
'Getafe': {'prov':'Madrid',
           'cod_est':'3195'},
'Málaga': {'prov':'Málaga',
           'cod_est':'6156X'},
'Villarreal': {'prov':'Castellón',
               'cod_est':'8500A'},
'Atlético de Madrid': {'prov':'Madrid',
                       'cod_est':'3195'},
'Real Sociedad': {'prov':'Gipuzkoa',
                  'cod_est':'1014A'},
'Las Palmas': {'prov':'Las Palmas',
               'cod_est':'C659M'},
'Celta de Vigo': {'prov':'Pontevedra',
         'cod_est':'1495'},
'Betis': {'prov':'Sevilla',
          'cod_est':'5783'},
'Valencia': {'prov':'Valencia',
             'cod_est':'8414A'}

}
